In [99]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from utils import *
from config import *
import numpy as np

In [100]:
dead_patients = load_processed_data('patient_dead', data_path=PROCESSED_ML_DATA_PATH)
dead_patients['Full Vaccination'] = np.where(dead_patients['admission_after_vaccine'] == 'Yes', 1, 0)
dead_patients = dead_patients.drop(columns=['admission_after_vaccine'])

In [101]:
dead_patients.nunique()

patient_num                57
encounter_num              57
dx_date_shifted            53
age_at_visit_years         37
payor_type_primary_name     9
admit_date_shifted         53
discharge_date_shifted     55
dx_code_list               57
total_length_of_stay       28
Alcohol Use Disorder        2
Cannabis Use Disorder       2
Cocaine Use Disorder        2
Opioid Use Disorder         2
Tobacco Use Disorder        2
Death                       1
Full Vaccination            2
dtype: int64

In [102]:
alive_patients = load_processed_data('patient_30_day_read', data_path=PROCESSED_ML_DATA_PATH)
alive_patients['Readmission'] = np.where(alive_patients['30_day_readmission'] == True, 1, 0).astype(int)
alive_patients = alive_patients.drop(columns=['30_day_readmission'])
alive_patients['Full Vaccination'] = np.where(alive_patients['admission_after_vaccine'] == 'Yes', 1, 0).astype(int)
alive_patients = alive_patients.drop(columns=['admission_after_vaccine'])

In [103]:
alive_patients.nunique()

patient_num                4435
encounter_num              7249
dx_date_shifted             990
age_at_visit_years           80
payor_type_primary_name      21
admit_date_shifted          990
discharge_date_shifted      989
dx_code_list               7137
total_length_of_stay         45
Alcohol Use Disorder          2
Cannabis Use Disorder         2
Cocaine Use Disorder          2
Opioid Use Disorder           2
Tobacco Use Disorder          2
Death                         1
Readmission                   2
Full Vaccination              2
dtype: int64

In [104]:
all_patients = pd.concat([dead_patients, alive_patients], axis=0)
all_patients.head()

,patient_num,encounter_num,dx_date_shifted,age_at_visit_years,payor_type_primary_name,admit_date_shifted,discharge_date_shifted,dx_code_list,total_length_of_stay,Alcohol Use Disorder,Cannabis Use Disorder,Cocaine Use Disorder,Opioid Use Disorder,Tobacco Use Disorder,Death,Full Vaccination,Readmission
0,17852216,115824905,2020-10-23,68,MEDICARE,2020-10-23,2020-11-02,"['G62.9', 'D69.6', 'Z66', 'J12.89', 'E88.09', ...",11,1,0,0,0,0,1,0,NaN
1,18344206,242777887,2021-09-27,77,Medicare HMO,2021-09-27,2021-09-27,"['Z87.891', 'R41.82', 'D64.9', 'I69.351', 'I10...",1,1,0,0,0,0,1,0,NaN
2,18431357,177586588,2022-01-26,68,Medicare HMO,2022-01-26,2022-01-27,"['U07.1', 'F17.210', 'R23.0', 'I46.9', 'R06.02']",2,0,0,0,0,1,1,0,NaN
3,18608899,117580339,2021-02-09,79,MEDICARE,2021-02-09,2021-02-11,"['E03.9', 'N17.9', 'G93.5', 'Z66', 'F17.210', ...",3,0,0,0,0,1,1,0,NaN
4,18721803,163588768,2021-01-04,75,Managed Care (Private),2021-01-04,2021-02-18,"['G93.89', 'E86.0', 'G93.41', 'R00.0', 'N17.9'...",46,1,0,0,0,0,1,0,NaN


In [105]:
patient_id_df = all_patients[['patient_num', 'encounter_num']]
patient_info_df = all_patients[['patient_num', 'encounter_num', 'admit_date_shifted', 'discharge_date_shifted']]

# Demographics

In [106]:
demographics_path = os.path.join(COVID_MERGED_DATA_PATH, 'demographics.csv')
demographics_copy = load_data(demographics_path)
demo_df = demographics_copy.copy()

In [107]:
demo_df.drop_duplicates(subset='patient_num', keep='first', inplace=True)

In [108]:
demo_df = demo_df[demographics_columns]

In [109]:
demo_df_merged = pd.merge(demo_df, patient_id_df, on='patient_num', how='inner')

In [110]:
demo_df_merged.nunique()

patient_num          4492
sex                     2
marital_status          8
employment_status      10
race                    8
ethnicity               5
zip3                   53
encounter_num        7306
dtype: int64

# BMI

In [111]:
vitals_path = os.path.join(COVID_MERGED_DATA_PATH, 'vitals.csv')
vitals_copy = load_data(vitals_path)

In [112]:
vitals_sort = vitals_copy.sort_values(by=['patient_num', 'bmi'], ascending=True, inplace=False)

In [113]:
# Calculate mean BMI for each patient 
bmi_means = vitals_sort.groupby('patient_num')['bmi'].mean().round(2)

In [114]:
bmi_df_merge = patient_id_df.merge(vitals_sort, on=['patient_num', 'encounter_num'], how='left')

In [115]:
# Fill NA values with patient mean
bmi_df_merge['bmi'] = bmi_df_merge['bmi'].fillna(bmi_df_merge['patient_num'].map(bmi_means))
bmi_df_merge['bmi'] = bmi_df_merge['bmi'].fillna(bmi_df_merge['bmi'].mean().round(2))

# Lab

In [116]:
diagnostic_results_path = os.path.join(COVID_MERGED_DATA_PATH, 'diagnostic_results.csv')
diagnostic_results_copy = load_data(diagnostic_results_path)
diagnostic_results_copy.shape

(27782275, 6)

In [117]:
diagnostic_results = diagnostic_results_copy.copy()
# drop PERFORMED BY in component_name
diagnostic_results.drop(diagnostic_results.loc[diagnostic_results['component_name']=='PERFORMED BY'].index, inplace=True)
# Drop rows with missing values in result_num
diagnostic_results.dropna(subset=['result_num'], inplace=True)

In [118]:
diagnostic_results

,patient_num,component_name,abnormal_ind,i2b2_date_shifted,result_num,result_modifier
0,22271013,GLUCOSE POC,AH,2021-03-28 21:37:00,141.000,EQ
1,19504545,MEAN CORPUSCULAR HGB CONCENTRN,NI,2017-07-09 13:28:00,33.100,EQ
2,23120699,BASOPHIL PERCENTAGE,NI,2021-01-05 07:17:00,0.000,EQ
3,30905545,SPECIFIC GRAVITY URINE,NI,2018-01-27 08:57:00,1.005,EQ
4,23304348,AST/SGOT,AH,2018-02-17 04:08:00,38.000,EQ
...,...,...,...,...,...,...
27782270,28211823,MONOCYTE ABSOLUTE,NI,2021-07-25 12:26:00,0.590,EQ
27782271,24560537,BKR CD AUTO MONO#,NI,2021-12-31 16:52:00,0.600,EQ
27782272,21941268,BKR ALANINE AMINOTRANSFERASE,NI,2022-02-26 10:15:00,7.000,EQ
27782273,17942470,LYMPH PERCENT,AL,2021-07-24 04:37:00,6.100,EQ


In [119]:
patient_info_df.nunique()

patient_num               4492
encounter_num             7306
admit_date_shifted         992
discharge_date_shifted     993
dtype: int64

In [120]:
diag_results_merge = patient_info_df.merge(diagnostic_results, how='left', on=['patient_num'])
diag_results_merge.nunique()

patient_num                 4492
encounter_num               7306
admit_date_shifted           992
discharge_date_shifted       993
component_name              3042
abnormal_ind                   9
i2b2_date_shifted         272136
result_num                 14334
result_modifier                5
dtype: int64

In [121]:
# Select patients with valid dates 
diag_results_merge['admit_date_shifted'] = pd.to_datetime(diag_results_merge['admit_date_shifted'],format='%Y-%m-%d')
diag_results_merge['discharge_date_shifted'] = pd.to_datetime(diag_results_merge['discharge_date_shifted'],format='%Y-%m-%d')
diag_results_merge['i2b2_date_shifted'] = pd.to_datetime(diag_results_merge['i2b2_date_shifted'],format='%Y-%m-%d %H:%M:%S')
diag_results_merge = diag_results_merge[(diag_results_merge['admit_date_shifted'] <= diag_results_merge['i2b2_date_shifted']) &  
                                        (diag_results_merge['i2b2_date_shifted'] <= diag_results_merge['admit_date_shifted'] + timedelta(days=14))]
diag_results_merge['patient_num'].nunique()

3850

In [122]:

diag_results_dedup = diag_results_merge.sort_values(by=['patient_num','admit_date_shifted','i2b2_date_shifted'
                                                        ]).drop_duplicates(subset=['patient_num','encounter_num','component_name'
                                                                                    ], keep='first')

In [123]:
# Create a pivot table based on component_name                                 
diag_results_subset = diag_results_dedup[['patient_num','encounter_num','component_name','result_num']]
df_pivot = diag_results_subset.pivot_table(index=['patient_num','encounter_num'],
                    columns='component_name', aggfunc='first').reset_index()
lab = pd.DataFrame(df_pivot.to_records())
lab.columns = [hdr.replace(", ''","").replace("('", "").replace("')", "").replace("result_num', '","") \
                    for hdr in lab.columns]
lab.shape

(6027, 1814)

In [124]:
#Drop features with small number of values
lst = drop_features(lab, 10)
lab.drop(columns=lst, inplace=True)
lab.drop(['index'], axis=1, inplace=True)
lab.fillna(0, inplace=True)  

Number of features with less than 10 values: 1060


In [125]:
lab.shape

(6027, 753)

In [126]:
lab.columns = ['lab_'+ str(col) for col in lab.columns]

In [127]:
lab

,lab_patient_num,lab_encounter_num,lab_% A1,lab_% F,lab_% IRON SATURATION,lab_% METHEMOGLOBIN,lab_% OXYHEMOGLOBIN,lab_% S,lab_25-75-75% PRE,lab_ABSOLUTE BASOPHILS,...,lab_VITAMIN E (BETA-GAMMA TOCOPHEROL),lab_VTG PRE,lab_WHITE BLOOD CELL COUNT,lab_WHITE BLOOD CELLS URINE,lab_WHOLE BLOOD CHLORIDE,lab_WHOLE BLOOD CREATININE,lab_WHOLE BLOOD GLUCOSE,lab_WHOLE BLOOD POTASSIUM,lab_WHOLE BLOOD SODIUM,lab_ZINC
0,16777685,144353253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16782332,209545972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,16784128,125498709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,16786590,103558812,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,12.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16786590,136360333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6022,65073452,175909283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6023,66672889,135967573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6024,66672889,135969484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6025,66672889,162891163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Merge All

In [128]:
df = all_patients.merge(demo_df_merged, on=['patient_num', 'encounter_num'], how='inner')

In [129]:
df = df.merge(bmi_df_merge, on=['patient_num', 'encounter_num'], how='inner')

In [130]:
final_data = df.merge(lab, left_on=['patient_num','encounter_num'],right_on=['lab_patient_num','lab_encounter_num'],how='inner')

In [131]:
final_data = get_longest_stay(final_data)

In [132]:
save_processed_data(final_data, 'final_data_all_columns', data_path=PROCESSED_ML_DATA_PATH)

In [133]:
neglect_cols = ['encounter_num',
                'admit_date_shifted', 'discharge_date_shifted', 'dx_date_shifted',
                'lab_patient_num', 'lab_encounter_num', 
                # 'language', 
                'patient_num',
                'dx_code_list'
]


In [134]:
final_data = final_data.drop(columns=neglect_cols, axis=1)
final_data = final_data.rename(columns={'sex': 'Sex', 'age_at_visit_years': 'Age'})

In [135]:
save_processed_data(final_data, 'final_data', data_path=PROCESSED_ML_DATA_PATH)

In [137]:
d = final_data[final_data['Death'] == 1]

In [138]:
d.shape

(57, 769)